In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import numpy as np
import mlflow
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import __version__ as sklearn_version

2024-10-25 11:03:35.818798: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-25 11:03:35.931633: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-25 11:03:35.978295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-25 11:03:36.074794: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-25 11:03:36.095882: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-25 11:03:36.215087: I tensorflow/core/platform/cpu_feature_gu

In [2]:
# Configuration du serveur MLflow
mlflow_server_uri = "https://champi.heuzef.com"
mlflow.set_tracking_uri(mlflow_server_uri)
mlflow.set_experiment("champi")

<Experiment: artifact_location='mlflow-artifacts:/103379370584144202', creation_time=1721579566179, experiment_id='103379370584144202', last_update_time=1721579566179, lifecycle_stage='active', name='champi', tags={'mlflow.sharedViewState.d4d3ab312f340bdf78e73ff3b2e223651ae813cd5c2bb0ee7938c7dba4de2511': '{"searchFilter":"","orderByKey":"attributes.start_time","orderByAsc":false,"startTime":"ALL","lifecycleFilter":"Active","datasetsFilter":[],"modelVersionFilter":"All '
                                                                                            'Runs","selectedColumns":["attributes.`Models`","metrics.`accuracy`","metrics.`val_accuracy`","attributes.`User`","params.`epochs`","attributes.`Description`"],"runsExpanded":{},"runsPinned":[],"runsHidden":[],"runsHiddenMode":"FIRST_10_RUNS","compareRunCharts":[{"uuid":"17218375793866mexilt9","type":"BAR","runsCountToCompare":10,"metricSectionId":"17218375793869x5p8wdz","deleted":false,"isGenerated":true,"metricKey":"train_accur

# Test et Evaluation du Modèle

In [3]:
# Charger le modèle
model = tf.keras.models.load_model('../../models/artifacts/yvan_jarvispore.h5')

model.summary()

/home/heuzef/GIT/jan24_cds_mushrooms/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/heuzef/GIT/jan24_cds_mushrooms/.venv/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 10, 10, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 10, 10, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 5, 5, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     6,554,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 23)             │        11,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,138,457 (31.05 MB)

 Trainable params: 8,136,471 (31.04 MB)

 Non-trainable params: 1,984 (7.75 KB)

 Optimizer params: 2 (12.00 B)

In [27]:
# Vérifier les noms des couches du modèle
for layer in model.layers:
    print(layer.name)

conv2d
batch_normalization
max_pooling2d
conv2d_1
batch_normalization_1
max_pooling2d_1
conv2d_2
batch_normalization_2
max_pooling2d_2
conv2d_3
batch_normalization_3
max_pooling2d_3
conv2d_4
batch_normalization_4
max_pooling2d_4
flatten
dense
dropout
dense_1


In [28]:
img_path = 'Stropharia_ambigua.jpg'
img = tf.keras.utils.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
# img_array /= 255.0 <= Ne pas appliquer cette normalisation !

print(img_array)

[[[[46. 55. 10.]
   [48. 61.  5.]
   [45. 70.  4.]
   ...
   [22. 34. 10.]
   [24. 34.  9.]
   [24. 34.  7.]]

  [[46. 63.  8.]
   [43. 68.  2.]
   [44. 71.  0.]
   ...
   [23. 38.  7.]
   [17. 32.  3.]
   [25. 39. 13.]]

  [[46. 68.  3.]
   [42. 72.  0.]
   [42. 72.  0.]
   ...
   [17. 31.  6.]
   [18. 30.  6.]
   [18. 30.  6.]]

  ...

  [[57. 93.  6.]
   [53. 93.  5.]
   [55. 92. 14.]
   ...
   [40. 20. 13.]
   [54. 25. 11.]
   [74. 27.  9.]]

  [[53. 91.  6.]
   [57. 94. 16.]
   [53. 84.  8.]
   ...
   [58. 18.  6.]
   [71. 28. 11.]
   [70. 24.  9.]]

  [[56. 88. 12.]
   [55. 87. 11.]
   [55. 90. 10.]
   ...
   [70. 39. 19.]
   [76. 42. 17.]
   [72. 37. 15.]]]]


In [29]:
predictions = model.predict(img_array)

predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[1.39423105e-15, 1.33577138e-09, 1.18861605e-11, 2.34543396e-13,
        1.46086858e-11, 6.81908577e-13, 1.31949776e-15, 6.21433040e-14,
        3.58152832e-03, 1.37310572e-05, 9.30511057e-10, 5.02229439e-11,
        3.02496773e-07, 1.34497757e-09, 1.17063070e-14, 1.64082564e-16,
        8.56261514e-13, 9.82579225e-14, 1.79859188e-16, 9.96404409e-01,
        5.06198165e-12, 1.81432788e-12, 1.04054815e-16]], dtype=float32)

In [30]:
score = tf.nn.softmax(predictions[0])

pd.DataFrame(tf.nn.softmax(predictions[0]))

,0
0,0.040466
1,0.040466
2,0.040466
3,0.040466
4,0.040466
5,0.040466
6,0.040466
7,0.040466
8,0.040611
9,0.040467


In [31]:
# Faire la prédiction
predictions = model.predict(img_array)

# Afficher la classe prédite (ou la distribution des probabilités)
predicted_class = np.argmax(predictions[0])  # Si le modèle a une sortie par classe
print(f'Classe prédite : {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Classe prédite : 19
